Requirements

In [16]:
# Install required libraries
!pip install torch torchvision torchaudio
!pip install numpy pandas scikit-learn
!pip install biopython

In [17]:
from Bio import SeqIO
from Bio.Seq import Seq
import os

# Log file setup
log_file_path = "cleaning_log.txt"
if os.path.exists(log_file_path):
    os.remove(log_file_path)
log_file = open(log_file_path, "w")

In [18]:
# Define file paths
fna_file = "Dataset\CCDS_nucleotide.current.fna"  
faa_file = "Dataset\CCDS_protein.current.faa"     
output_clean_fna = "Dataset\cleaned_nucleotide_file.fna"
output_clean_faa = "Dataset\cleaned_protein_file.faa"

In [19]:
# Function to parse and validate FASTA sequences
def parse_fasta(file_path, file_type="fna"):
    sequences = {}
    for record in SeqIO.parse(file_path, "fasta"):
        seq = str(record.seq).upper()  # Normalize to uppercase
        if file_type == "fna" and all(base in "ATGC" for base in seq):  # Validate nucleotides
            sequences[record.id] = seq
        elif file_type == "faa" and all(base in "ACDEFGHIKLMNPQRSTVWY*" for base in seq):  # Validate proteins
            sequences[record.id] = seq
        else:
            log_file.write(f"Invalid sequence in {file_type}: {record.id}\n")
    return sequences

# Function to clean and match FASTA sequences
def clean_and_match_fasta(fna_sequences, faa_sequences):
    cleaned_fna = {}
    cleaned_faa = {}

    for seq_id in faa_sequences:
        if seq_id in fna_sequences:
            nucleotide_seq = Seq(fna_sequences[seq_id])
            try:
                # Translate the nucleotide sequence
                translated_seq = nucleotide_seq.translate(to_stop=True)
                # Check if translation matches the protein sequence
                if str(translated_seq) == faa_sequences[seq_id]:
                    cleaned_fna[seq_id] = fna_sequences[seq_id]
                    cleaned_faa[seq_id] = faa_sequences[seq_id]
                else:
                    log_file.write(f"Translation mismatch for ID {seq_id}\n")
            except Exception as e:
                log_file.write(f"Translation error for ID {seq_id}: {e}\n")
        else:
            log_file.write(f"Missing nucleotide sequence for protein ID {seq_id}\n")

    # Log any nucleotide sequences with no matching protein
    for seq_id in fna_sequences:
        if seq_id not in faa_sequences:
            log_file.write(f"Unmatched nucleotide sequence ID {seq_id}\n")

    return cleaned_fna, cleaned_faa

# Function to save cleaned FASTA sequences
def save_cleaned_sequences(cleaned_sequences, output_file):
    records = []
    for seq_id, sequence in cleaned_sequences.items():
        record = SeqIO.SeqRecord(Seq(sequence), id=seq_id, description="")
        records.append(record)
    SeqIO.write(records, output_file, "fasta")

In [20]:
# Load nucleotide and protein sequences
nucleotide_sequences = parse_fasta(fna_file, file_type="fna")
protein_sequences = parse_fasta(faa_file, file_type="faa")
print(f"Loaded {len(nucleotide_sequences)} nucleotide sequences.")
print(f"Loaded {len(protein_sequences)} protein sequences.")

Loaded 35624 nucleotide sequences.
Loaded 35573 protein sequences.


In [21]:
# Clean and match sequences
cleaned_nucleotide, cleaned_protein = clean_and_match_fasta(nucleotide_sequences, protein_sequences)
print(f"Cleaned {len(cleaned_nucleotide)} matching nucleotide sequences.")
print(f"Cleaned {len(cleaned_protein)} matching protein sequences.")

c:\Users\Owner\AppData\Local\Programs\Python\Python311\Lib\site-packages\Bio\Seq.py:2879: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


Cleaned 35510 matching nucleotide sequences.
Cleaned 35510 matching protein sequences.


In [42]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

# Function to save cleaned sequences to a file
def save_cleaned_sequences(cleaned_sequences, output_file):
    records = []
    for i, sequence in enumerate(cleaned_sequences):
        record = SeqRecord(Seq("".join(map(str, sequence))), id=f"seq{i+1}", description="")
        records.append(record)
    SeqIO.write(records, output_file, "fasta")

# Clean and match sequences
cleaned_nucleotide, cleaned_protein = clean_and_match_fasta(nucleotide_sequences, protein_sequences)
print(f"Cleaned {len(cleaned_nucleotide)} matching nucleotide sequences.")
print(f"Cleaned {len(cleaned_protein)} matching protein sequences.")

# Function to encode nucleotide sequences
def encode_nucleotide_sequence(sequence):
    mapping = {'A': 0, 'T': 1, 'G': 2, 'C': 3}
    encoded_sequence = []
    for base in sequence:
        if base in mapping:
            encoded_sequence.append(mapping[base])
        else:
            print(f"Skipping unexpected character '{base}' in sequence.")
    return encoded_sequence

# Function to encode protein sequences
def encode_protein_sequence(sequence):
    mapping = {
        'A': 0, 'R': 1, 'N': 2, 'D': 3, 'C': 4, 'E': 5, 'Q': 6, 'G': 7, 'H': 8, 'I': 9,
        'L': 10, 'K': 11, 'M': 12, 'F': 13, 'P': 14, 'S': 15, 'T': 16, 'W': 17, 'Y': 18, 'V': 19
    }
    encoded_sequence = []
    for amino_acid in sequence:
        if amino_acid in mapping:
            encoded_sequence.append(mapping[amino_acid])
        else:
            print(f"Skipping unexpected character '{amino_acid}' in sequence.")
    return encoded_sequence

# Function to filter and encode sequences
def filter_and_encode_sequences(sequences, encode_function):
    encoded_sequences = []
    for seq in sequences:
        if not seq.startswith('>') and not any(char.isdigit() for char in seq):
            encoded_sequences.append(encode_function(seq))
    return encoded_sequences

# Encode cleaned nucleotide and protein sequences
encoded_nucleotide = filter_and_encode_sequences(cleaned_nucleotide, encode_nucleotide_sequence)
encoded_protein = filter_and_encode_sequences(cleaned_protein, encode_protein_sequence)

# Save cleaned sequences to files
save_cleaned_sequences(encoded_nucleotide, output_clean_fna)
save_cleaned_sequences(encoded_protein, output_clean_faa)
print(f"Cleaned nucleotide sequences saved to: {output_clean_fna}")
print(f"Cleaned protein sequences saved to: {output_clean_faa}")
log_file.close()

ValueError: I/O operation on closed file.

In [11]:
import pandas as pd
import torch

codon_usage = pd.read_csv("Dataset\\o537-Tissue_Codon.tsv", sep='\t', index_col=0)
bicodon_usage = pd.read_csv("Dataset\\o537-Tissue_Bicod.tsv", sep='\t', index_col=0)

# Function to encode codons and bicodons
def encode_sequence(sequence):
    mapping = {'A': 0, 'T': 1, 'G': 2, 'C': 3}
    encoded_sequence = []
    for base in sequence:
        if base in mapping:
            encoded_sequence.append(mapping[base])
        else:
            raise ValueError(f"Unexpected character '{base}' in sequence.")
    return encoded_sequence

# Remove the last column in both codon usage and bicodon usage
codon_usage = codon_usage.iloc[:, :-1]
bicodon_usage = bicodon_usage.iloc[:, :-1]

# Encode tissue types (rows) - shared between codon and bicodon datasets
tissue_names = codon_usage.index.tolist()  # Assume same tissues for both datasets
tissue_encoder = {tissue: idx for idx, tissue in enumerate(tissue_names)}
encoded_tissues = [tissue_encoder[tissue] for tissue in tissue_names]

# Print codon columns to debug
print("Codon columns:", codon_usage.columns.tolist())

# Encode codons (columns for codon usage)
encoded_codons = [encode_sequence(codon) for codon in codon_usage.columns]

# Encode bicodons (columns for bicodon usage)
encoded_bicodons = [encode_sequence(bicodon[:3]) + encode_sequence(bicodon[3:]) for bicodon in bicodon_usage.columns]

# Convert dataframes to feature tensors
codon_feature_matrix = torch.tensor(codon_usage.values, dtype=torch.float32)
bicodon_feature_matrix = torch.tensor(bicodon_usage.values, dtype=torch.float32)

# Convert encoded tissues, codons, and bicodons into tensors
tissue_tensor = torch.tensor(encoded_tissues)
codon_tensor = torch.tensor(encoded_codons)
bicodon_tensor = torch.tensor(encoded_bicodons)

# Display results
print("Tissue indices:", tissue_tensor.shape, tissue_tensor)
print("Encoded codons:", codon_tensor.shape, codon_tensor[:5])  # Show first 5
print("Encoded bicodons:", bicodon_tensor.shape, bicodon_tensor[:5])  # Show first 5
print("Codon feature tensor shape:", codon_feature_matrix.shape)
print("Bicodon feature tensor shape:", bicodon_feature_matrix.shape)

Codon columns: ['TTT', 'TTC', 'TTA', 'TTG', 'CTT', 'CTC', 'CTA', 'CTG', 'ATT', 'ATC', 'ATA', 'ATG', 'GTT', 'GTC', 'GTA', 'GTG', 'TAT', 'TAC', 'TAA', 'TAG', 'CAT', 'CAC', 'CAA', 'CAG', 'AAT', 'AAC', 'AAA', 'AAG', 'GAT', 'GAC', 'GAA', 'GAG', 'TCT', 'TCC', 'TCA', 'TCG', 'CCT', 'CCC', 'CCA', 'CCG', 'ACT', 'ACC', 'ACA', 'ACG', 'GCT', 'GCC', 'GCA', 'GCG', 'TGT', 'TGC', 'TGA', 'TGG', 'CGT', 'CGC', 'CGA', 'CGG', 'AGT', 'AGC', 'AGA', 'AGG', 'GGT', 'GGC', 'GGA', 'GGG']
Tissue indices: torch.Size([53]) tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52])
Encoded codons: torch.Size([64, 3]) tensor([[1, 1, 1],
        [1, 1, 3],
        [1, 1, 0],
        [1, 1, 2],
        [3, 1, 1]])
Encoded bicodons: torch.Size([4096, 6]) tensor([[1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 3],
        [1, 1, 1, 1, 1, 0],
        [1

In [14]:
# Normalize the codon and bicodon tensors
def normalize_tensor(tensor):
    mean = tensor.mean(dim=0, keepdim=True)
    std = tensor.std(dim=0, keepdim=True)
    return (tensor - mean) / std

# Normalize the codon and bicodon feature matrices
normalized_codon_feature_matrix = normalize_tensor(codon_feature_matrix)
normalized_bicodon_feature_matrix = normalize_tensor(bicodon_feature_matrix)

# Concatenate the normalized codon and bicodon feature matrices along the feature dimension
combined_feature_matrix = torch.cat((normalized_codon_feature_matrix, normalized_bicodon_feature_matrix), dim=1)

# Display normalized and combined results
print("Normalized codon feature tensor shape:", normalized_codon_feature_matrix.shape)
print("Normalized bicodon feature tensor shape:", normalized_bicodon_feature_matrix.shape)
print("Combined feature tensor shape:", combined_feature_matrix.shape)

Normalized codon feature tensor shape: torch.Size([53, 64])
Normalized bicodon feature tensor shape: torch.Size([53, 4096])
Combined feature tensor shape: torch.Size([53, 4160])
